In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from emulation import Emulation

In [122]:
import yaml
import pprint
import pyperclip as clip

def printLogs(logs):
    logs_str = yaml.dump(logs, default_flow_style=False)
    clip.copy(f"{logs}")
    print(logs_str)
    print('=========================')

def copyLogs(logs):
    clip.copy(f"{logs}")

### Памп и перезакладывание

In [165]:
def gigaFlip():
    exp = Emulation()
    log_start = exp.logState()
    
    prevBprice = exp.router.prices[exp.tokens[1]]

    exp.host.mintTokens(exp.tokens[:2], [100, 100])

    exp.host.deposit(0, exp.tokens[:2], [100, 100])

    exp.farms.block += 60
    
    exp.router.changeTokenPrice(exp.tokens[1], prevBprice*100)

    exp.host.withdraw(0)

    exp.router.changeTokenPrice(exp.tokens[1], prevBprice)
    
    exp.host.deposit(0, exp.tokens[:2], [exp.tokens[0].balanceOf(exp.host), exp.tokens[1].balanceOf(exp.host)])

    exp.farms.block += 36000

    log_end = exp.logState()

    for pid in exp.farms.userInfo:
        for user in exp.farms.userInfo[pid]:
            user.withdraw(pid)
            print(f"{user.name} balance: {exp.doku.balanceOf(user)//1e18}")
    
    return log_start, log_end

def allOk():
    exp = Emulation()
    log_start = exp.logState()
    
    prevBprice = exp.router.prices[exp.tokens[1]]

    exp.host.mintTokens(exp.tokens[:2], [100, 100])

    exp.host.deposit(0, exp.tokens[:2], [100, 100])

    exp.farms.block += 60
    exp.router.changeTokenPrice(exp.tokens[1], prevBprice*100)

    exp.router.changeTokenPrice(exp.tokens[1], prevBprice)
    
    exp.farms.block += 36000

    log_end = exp.logState()

    for pid in exp.farms.userInfo:
        for user in exp.farms.userInfo[pid]:
            user.withdraw(pid)
            print(f"{user.name} balance: {exp.doku.balanceOf(user)//1e18}")
    
    return log_start, log_end
    
    

In [166]:
log_start, log_end = gigaFlip()
print('===================')
_, _ = allOk()

alice balance: 10462.0
HOST balance: 22283.0
bob balance: 16230.0
carol balance: 19813.0
david balance: 30275.0
alice balance: 15025.0
HOST balance: 15025.0
bob balance: 14364.0
carol balance: 19813.0
david balance: 34838.0


In [167]:
copyLogs(log_end)

In [155]:
log_start, log_end = gigaFlip()
print('===================')
_, _ = allOk()

alice balance: 10462.0
HOST balance: 22283.0
bob balance: 16230.0
carol balance: 19813.0
david balance: 30275.0
alice balance: 15025.0
HOST balance: 15025.0
bob balance: 14364.0
carol balance: 19813.0
david balance: 34838.0


In [124]:
copyLogs(log_start)

In [125]:
copyLogs(log_end)

### Цена токена растёт в 2 раза, при каждом повышении HOST перезакладывает

In [147]:
def reBasing():
    exp = Emulation()

    exp.host.mintTokens(exp.tokens[:2], [100, 100])

    exp.host.deposit(0, exp.tokens[:2], [100, 100])

    exp.farms.block += 60
    
    nIter = 100
    for i in range(nIter):
        log_prev = exp.logState()
        prevBprice = exp.router.prices[exp.tokens[1]]
        exp.router.changeTokenPrice(exp.tokens[1], prevBprice * 2**(1/nIter))
        exp.host.withdraw(0)    
        exp.host.deposit(0, exp.tokens[:2], [exp.tokens[0].balanceOf(exp.host), exp.tokens[1].balanceOf(exp.host)])
        exp.farms.block += 60

    log_last = exp.logState()

    for pid in exp.farms.userInfo:
        for user in exp.farms.userInfo[pid]:                
            user.withdraw(pid)
            print(f"{user.name} balance: {exp.doku.balanceOf(user)//1e18}")
    
    return log_prev, log_last

def allOk():
    exp = Emulation()

    exp.host.mintTokens(exp.tokens[:2], [100, 100])

    exp.host.deposit(0, exp.tokens[:2], [100, 100])

    exp.farms.block += 60
    
    nIter = 100
    for i in range(nIter):
        log_prev = exp.logState()
        prevBprice = exp.router.prices[exp.tokens[1]]
        exp.router.changeTokenPrice(exp.tokens[1], prevBprice * 2**(1/nIter))
        exp.farms.block += 60

    log_last = exp.logState()

    for pid in exp.farms.userInfo:
        for user in exp.farms.userInfo[pid]:                
            user.withdraw(pid)
            print(f"{user.name} balance: {exp.doku.balanceOf(user)//1e18}")
    
    return log_prev, log_last

In [148]:
log_start, log_end = reBasing()
print('============')
_, _ = allOk()

alice balance: 2482.0
HOST balance: 2565.0
bob balance: 2458.0
carol balance: 3329.0
david balance: 5812.0
alice balance: 2525.0
HOST balance: 2525.0
bob balance: 2414.0
carol balance: 3329.0
david balance: 5854.0


In [ ]:
copyLogs(log_start)

In [ ]:
copyLogs(log_end)